In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                    options=[
                                            {'label': 'All Sites', 'value': 'ALL'},
                                            {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                            {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                            {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                            {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                    ],
                                        value= 'ALL',
                                        placeholder='Select a Launch Site Here',
                                        searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={
                                                    0: '0 Kg',
                                                    2500: '2500 Kg',
                                                    5000: '5000 Kg',
                                                    7500: '7500 Kg',
                                                    10000: '10000 Kg',
                                                    },
                                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class', names='Launch Site', title='Success Launches for All Sites')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site].groupby(['Launch Site', 'class']). \
        size().reset_index(name='class count')
        title = f"Total Success Launches for site {entered_site}"
        fig = px.pie(filtered_df,values='class count', names='class', title=title)
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
             [Input(component_id='site-dropdown', component_property='value'),
              Input(component_id='payload-slider', component_property='value')]
              )
def get_scatter_chart(entered_site, slider_range):
    low, high = slider_range
    filtered_df_s = spacex_df[(spacex_df['Payload Mass (kg)'] > low) & (spacex_df['Payload Mass (kg)'] < high)]
    if entered_site == 'ALL':
        scatter_fig = px.scatter(filtered_df_s, x="Payload Mass (kg)", y="class", color="Booster Version Category",
        title='Correlation between Payload and Success for all Sites')
        return scatter_fig
    else:
        filtered_df_s_s = filtered_df_s[(filtered_df_s['Launch Site'] == entered_site)]
        scatter_fig = px.scatter(filtered_df_s_s, x="Payload Mass (kg)", y="class", color="Booster Version Category",
        title = f"Correlation between Payload and Success for Site {entered_site}")
        return scatter_fig

# Run the app
if __name__ == '__main__':
    app.run_server()


### Success Launches for All Sites  
<img src="PlotlyDash-SuccessLaunchesForAllSites.png" width=1000 height=1000 />

### Total Success Launches For Site CCAFS LC-40  
<img src="PlotlyDash-TotalSuccessLaunchesForSiteCCAFS_LC-40.png" width=1000 height=1000 />

### Total Success Launches For Site VAFB SLC-4E  
<img src="PlotlyDash-TotalSuccessLaunchesForSiteVAFB_SLC-4E.png" width=1000 height=1000 />

### Total Success Launches For Site KSC LC-39A  
<img src="PlotlyDash-TotalSuccessLaunchesForSiteKSC_LC-39A.png" width=1000 height=1000 />

### Total Success Launches For Site CCAFS SLC-40  
<img src="PlotlyDash-TotalSuccessLaunchesForSiteCCAFS_SLC-40.png" width=1000 height=1000 />

## Payload vs. Launch Outcome for all sites - Payload beyween 0 and 5000
<img src="PlotlyDash-DashboardComplete_0-5000.png" width=1000 height=1000 />

## Payload vs. Launch Outcome for all sites - Payload beyween 2500 and 7500
<img src="PlotlyDash-DashboardComplete_2500-7500.png" width=1000 height=1000 />

## Payload vs. Launch Outcome for all sites - Payload beyween 5000 and 10000
<img src="PlotlyDash-DashboardComplete_5000-10000.png" width=1000 height=1000 />

## Payload vs. Launch Outcome for all sites - All Payloads
<img src="PlotlyDash-DashboardComplete_small.png" width=1000 height=1000 />

## Payload vs. Launch Outcome for all sites - Payload beyween 2000 and 6000
<img src="PlotlyDash-DashboardComplete_2000-6000.png" width=1000 height=1000 />